# Gerekli Kütüphaneleri içe aktarma

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn import neighbors
from sklearn.svm import SVR


# Veri Setini Yükleme

In [ ]:
#veri setini yükle
t_data = pd.read_csv('./sample_data/train_data.csv')
p_data = pd.read_csv('./sample_data/predict_data.csv')


# Özellik Mühendisliği

In [ ]:
'''
train_data["Age"] = 2024 - train_data["yr_built"]
train_data["Renovated"] = train_data["yr_renovated"].apply(lambda x: 1 if x>0 else 0)
train_data["bedrooms/sqft_living"] = train_data["bedrooms"] / train_data["sqft_living"]
train_data["bathrooms/sqft_living"] = train_data["bathrooms"] / train_data["sqft_living"]

predict_data["Age"] = 2024 - predict_data["yr_built"]
predict_data["Renovated"] = predict_data["yr_renovated"].apply(lambda x: 1 if x> 0 else 0)
predict_data["bedrooms/sqft_living"] = predict_data["bedrooms"] / predict_data["sqft_living"]
predict_data["bathrooms/sqft_living"] = predict_data["bathrooms"] / predict_data["sqft_living"]

SyntaxError: incomplete input (<ipython-input-33-d1c813a670ff>, line 1)

In [ ]:
t_data.drop(columns = ["date","country", "street","statezip","city"], axis = 1, inplace = True)

In [ ]:
p_data.drop(columns = ["date", "street", "country","city", "statezip"], axis = 1, inplace = True)

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4140 entries, 0 to 4139
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          4140 non-null   float64
 1   bedrooms       4140 non-null   float64
 2   bathrooms      4140 non-null   float64
 3   sqft_living    4140 non-null   int64  
 4   sqft_lot       4140 non-null   int64  
 5   floors         4140 non-null   float64
 6   waterfront     4140 non-null   int64  
 7   view           4140 non-null   int64  
 8   condition      4140 non-null   int64  
 9   sqft_above     4140 non-null   int64  
 10  sqft_basement  4140 non-null   int64  
 11  yr_built       4140 non-null   int64  
 12  yr_renovated   4140 non-null   int64  
dtypes: float64(4), int64(9)
memory usage: 420.6 KB


In [ ]:
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bedrooms       460 non-null    float64
 1   bathrooms      460 non-null    float64
 2   sqft_living    460 non-null    int64  
 3   sqft_lot       460 non-null    int64  
 4   floors         460 non-null    float64
 5   waterfront     460 non-null    int64  
 6   view           460 non-null    int64  
 7   condition      460 non-null    int64  
 8   sqft_above     460 non-null    int64  
 9   sqft_basement  460 non-null    int64  
 10  yr_built       460 non-null    int64  
 11  yr_renovated   460 non-null    int64  
dtypes: float64(3), int64(9)
memory usage: 43.2 KB


# Normalizasyon ve Model Eğitimi



In [ ]:
X = t_data.drop(["price"], axis = 1)
y = t_data["price"]

In [ ]:
ss = StandardScaler()
X = ss.fit_transform(X)

ss = StandardScaler()
p_data = ss.fit_transform(p_data)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
rf_model = RandomForestRegressor(random_state = 42).fit(X_train,y_train)
y_pred = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, y_pred)

In [ ]:
dt_model = DecisionTreeRegressor(random_state=42)
y_pred = rf_model.predict(X_test)
dt_mse = mean_squared_error(y_test, y_pred)

In [ ]:
lr_model = LinearRegression()
y_pred = rf_model.predict(X_test)
lr_mse = mean_squared_error(y_test, y_pred)

In [ ]:
gb_model = GradientBoostingRegressor(random_state=42)
y_pred = rf_model.predict(X_test)
gb_mse = mean_squared_error(y_test, y_pred)

In [ ]:
xgb_model =XGBRegressor(random_state=42)
y_pred = rf_model.predict(X_test)
xgb_mse = mean_squared_error(y_test, y_pred)

In [ ]:
lgbm_model = LGBMRegressor(random_state=42)
y_pred = rf_model.predict(X_test)
lgbm_mse = mean_squared_error(y_test, y_pred)

In [ ]:
mse_values = {rf_model: rf_mse, dt_model: dt_mse, lr_model: lr_mse, gb_model: gb_mse, xgb_model: xgb_mse, lgbm_model: lgbm_mse}
best_model = min(mse_values, key=mse_values.get)
print(f"The best model is {best_model} with MSE: {mse_values[best_model]}")

The best model is RandomForestRegressor(random_state=42) with MSE: 100486296277718.7


In [ ]:
from sklearn.model_selection import RandomizedSearchCV


rf_params = {"max_depth":[5,8,10],
             "max_features":[2,5,10],
             "n_estimators":[200,500,1000,2000],
             "min_samples_split":[2,10,80,100]}

rf_cv_model = RandomizedSearchCV(rf_model, rf_params, n_iter=20, cv=5, n_jobs=-1, verbose=2, random_state=42).fit(X_train, y_train)
rf_cv_model.best_params_

Fitting 5 folds for each of 20 candidates, totalling 100 fits


{'n_estimators': 200,
 'min_samples_split': 100,
 'max_features': 2,
 'max_depth': 5}

In [ ]:
rf_model =RandomForestRegressor(
    max_depth=5,
    max_features=2,
    min_samples_split=100,
    n_estimators=200,
    random_state=42
    )

rf_model.fit(X_train,y_train)
y_pred = rf_model.predict(X_test)
mean_squared_error(y_test, y_pred)

97286579084472.64

In [ ]:
testing_predict = rf_model.predict(p_data)

In [ ]:
Id = []
for i in range(len(p_data)) :
    Id.append(i)

In [ ]:
output = pd.DataFrame({"ID" : Id,
                      "price" : testing_predict})
output

,ID,price
0,0,5.567708e+05
1,1,5.929810e+05
2,2,4.266913e+05
3,3,1.369656e+06
4,4,4.489271e+05
...,...,...
455,455,6.745188e+05
456,456,3.903657e+05
457,457,5.005687e+05
458,458,9.460488e+05


In [ ]:
output.to_csv("submission_rf_model.csv",index = False)